# Agent to Tailor Job Application

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [2]:
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from utils import get_openai_api_key, get_serper_api_key
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

import os


c:\Users\sarth\crewai\.conda\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\sarth\crewai\.conda\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
c:\Users\sarth\crewai\.conda\Lib\site-packages\crewai_tools\tools\selenium_scraping_tool\selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic 

In [3]:
load_dotenv()

os.environ['OPENAI_MODEL_NAME'] = 'gpt-3.5-turbo-0125'
os.environ['SERPER_API_KEY'] = get_serper_api_key()
os.environ['OPENAI_API_KEY'] = get_openai_api_key()


In [6]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.55s/it]


In [7]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [8]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [9]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [10]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [11]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

In [12]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [13]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [14]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [15]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


In [16]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

In [17]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [18]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c6620af9-aac4-4216-b969-92f34e5ec006                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8acd15cf-6514-47bd-9db5-2ce6af6e99f6
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: 8acd15cf-6514-47bd-9db5-2ce6af6e99f6
│      Status: Executing Task...
└── 📋 Task: 8f210d39-5949-479b-bcec-1b9d1facca74
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: 8acd15cf-6514-47bd-9db5-2ce6af6e99f6
│      Status: Executing Task...
└── 📋 Task: 8f210d39-5949-479b-bcec-1b9d1facca74
       Status: Executing Task...
    └── 🤖 Agent: Tech Job Researcher
            Status: In Progress

# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.


...



# Agent: Engineering Interview Preparer
## Final Answer: 
## Interview Questions and Talking Points for Noah Williams:

### Leadership and Team Management:
1. Can you provide an example of a challenging situation you faced while managing remote and in-office engineering teams? How did you approach and resolve it?
2. How do you ensure effective collaboration and communication within your teams, especially when working across different time zones?
3. What strategies do you implement to foster a culture of continuous improvement and high performance among your team members?
4. Discuss a successful project where you demonstrated strong leadership skills and the impact it had on the team and the project outcome.

### Technical Expertise and Innovation:
1. In your role as a Senior Software Engineer at DriveAI, you developed and optimized a central API. Can you elaborate on the challenges you encountered and the solutions you implemented to improve functionality?
2. As the CTO at BetCraft, 

🚀 Crew: crew
├── 📋 Task: 8acd15cf-6514-47bd-9db5-2ce6af6e99f6
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
├── 📋 Task: 8f210d39-5949-479b-bcec-1b9d1facca74
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: In Progress
│   │   └── 🧠 Thinking...
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: ✅ Completed
│   │   ├── 🔧 Using Read website content (2)
│   │   ├── 🔧 Used Search the internet with Serper (1)
│   │   ├── 🔧 Used Read website content (2)
│   │   └── 🔧 Used Read a file's content (1)
│   └── 🤖 Agent: Personal Profiler for Engineers
│           Status: ✅ Completed
│       ├── 🔧 Used Read website content (3)
│       └── 🔧 Used Search the internet with Serper (2)
├── 📋 Task: 560e7f55-631f-4462-85a9-3df9e60da0d8
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
│       └── 🔧 Used Search a MDX's content (1)
└── 📋 Task: 7e8325b5-34f7-4261-ad00-47fa293a6abf
       Status: Executing Task...
    └── 🤖 Agent: Engineering Interview Preparer
            Status: ✅ Completed
        ├── 🔧 Used Read a file's content (2)
        └── 🔧 Used Search a MDX's content (3)

🚀 Crew: crew
├── 📋 Task: 8acd15cf-6514-47bd-9db5-2ce6af6e99f6
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
├── 📋 Task: 8f210d39-5949-479b-bcec-1b9d1facca74
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: In Progress
│   │   └── 🧠 Thinking...
│   ├── 🤖 Agent: Tech Job Researcher
│   │       Status: ✅ Completed
│   │   ├── 🔧 Using Read website content (2)
│   │   ├── 🔧 Used Search the internet with Serper (1)
│   │   ├── 🔧 Used Read website content (2)
│   │   └── 🔧 Used Read a file's content (1)
│   └── 🤖 Agent: Personal Profiler for Engineers
│           Status: ✅ Completed
│       ├── 🔧 Used Read website content (3)
│       └── 🔧 Used Search the internet with Serper (2)
├── 📋 Task: 560e7f55-631f-4462-85a9-3df9e60da0d8
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
│       └── 🔧 Used Search a MDX's content (1)
└── 📋 Task: 7e8325b5-34f7-4261-ad00-47fa293a6abf
       Assigned to: Engineering Interview Preparer
       Status: ✅ Completed
    └── 🤖 Agent: Engineering Interview Preparer
            Status: ✅ Completed
        ├── 🔧 Used Read a file's content (2)
        └── 🔧 Used Search a MDX's content (3)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7e8325b5-34f7-4261-ad00-47fa293a6abf                                                                     │
│  Agent: Engineering Interview Preparer                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c6620af9-aac4-4216-b969-92f34e5ec006                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [19]:
print(result)

## Interview Questions and Talking Points for Noah Williams:

### Leadership and Team Management:
1. Can you provide an example of a challenging situation you faced while managing remote and in-office engineering teams? How did you approach and resolve it?
2. How do you ensure effective collaboration and communication within your teams, especially when working across different time zones?
3. What strategies do you implement to foster a culture of continuous improvement and high performance among your team members?
4. Discuss a successful project where you demonstrated strong leadership skills and the impact it had on the team and the project outcome.

### Technical Expertise and Innovation:
1. In your role as a Senior Software Engineer at DriveAI, you developed and optimized a central API. Can you elaborate on the challenges you encountered and the solutions you implemented to improve functionality?
2. As the CTO at BetCraft, you led significant technological advancements. How did you 